In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

# Import scripts
from script_files.feature_engineering import *
from script_files.prepare_flight_data import *
from script_files.dummies import *

In [2]:
train_df = pd.read_csv('data/final_train.csv', index_col=[0])
train_df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,delayed,domestic,dep_hour,dep_weekday,duration_min,arr_hour,flight_month,flight_month_name,year,distance
0,train_id_15674,2016-01-01 00:00:00,TU 0564,NKC,TUN,2016-01-01 00:15:00,2016-01-01 04:30:00,ATA,TU 320IMV,0.0,...,0,0,0,Friday,255.0,4,1,January,2016,3298.067996
1,train_id_15676,2016-01-01 00:00:00,TU 0714,JED,TUN,2016-01-01 00:55:00,2016-01-01 05:30:00,ATA,TU 332IFM,195.0,...,1,0,0,Friday,275.0,5,1,January,2016,3256.052105
2,train_id_15675,2016-01-01 00:00:00,TU 0614,DKR,TUN,2016-01-01 01:20:00,2016-01-01 05:55:00,ATA,TU 320IMU,49.0,...,1,0,1,Friday,275.0,5,1,January,2016,3678.974557
3,train_id_30980,2016-01-01 00:00:00,UG 0002,TUN,DJE,2016-01-01 06:15:00,2016-01-01 07:15:00,SCH,UG AT7LBD,0.0,...,0,1,6,Friday,60.0,7,1,January,2016,333.916459
4,train_id_7179,2016-01-01 00:00:00,TU 0880,TUN,AMS,2016-01-01 06:30:00,2016-01-01 09:20:00,ATA,TU 736IOP,36.0,...,1,0,6,Friday,170.0,9,1,January,2016,1770.371959


In [3]:
y_pred = dummie_reg(train_df)
y_pred

[89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
 89.38,
